# Cold Start Baselines

**ab_240422** appeared to have the same performance as the previous **best_meme_from_each_source**. The reasons why these results differ from the previous offline simulation

* The main probable reason is the bias in the training data
* In the previous simulation, I removed users that blocked the bot
* In the current simulation I left only to 40 items for each new user

In [1]:
import hashlib
import json
from datetime import datetime

import numpy as np
import polars as pl
from sklearn.model_selection import train_test_split

In [2]:
from models import BaseRecommender, BestMemeFromEachSource, MostLiked
from estimate import estimate, estimate_prod

In [3]:
coldstart_df = pl.read_parquet('coldstart_dataset.pq')
meme_features_daily_df = pl.read_parquet('meme_features_daily.pq')

# v2 is calculated on users with less than 200 responses
meme_features_daily_v2_df = pl.read_parquet('meme_features_daily_v2.pq')

In [4]:
coldstart_df.slice(2, 1)

user_id,hist_size,dtm,date_dtm,hist_memes,hist_reactions,target_memes,target_reactions,target_recommended_by
str,i64,datetime[μs],datetime[μs],list[str],list[i64],list[str],list[i64],list[str]
"""486,191,407""",20,2024-04-03 19:20:00,2024-04-03 00:00:00,"[""1,237,876"", ""2,829,942"", … ""3,755,263""]","[2, 2, … 1]","[""1,197,484"", ""3,546,640"", … ""2,051,741""]","[2, 2, … 2]","[""like_spread_and_recent"", ""like_spread_and_recent"", … ""best_meme_from_each_source""]"


In [5]:
validation_df = (
    coldstart_df
    .filter(pl.col('date_dtm') >= datetime(2024, 4, 25))
    .filter(pl.col('date_dtm') < datetime(2024, 5, 5))
    .filter(pl.col('hist_size') <= 30)
)

validation_2_df = (
    coldstart_df
    .filter(pl.col('date_dtm') >= datetime(2024, 4, 25))
    .filter(pl.col('date_dtm') < datetime(2024, 5, 5))
    .filter(pl.col('hist_size') <= 30)
    .with_columns(pl.col('target_memes').list.head(10))
    .with_columns(pl.col('target_reactions').list.head(10))
    .with_columns(pl.col('target_recommended_by').list.head(10))
)

In [6]:
estimate_prod('best_meme_from_each_source', validation_df)

Likes - 33110, Like Rate = 0.533 +- 0.030, Like Rate Micro = 0.437 +- 0.016


In [7]:
estimate_prod('best_meme_from_each_source', validation_2_df)

Likes - 5068, Like Rate = 0.520 +- 0.018, Like Rate Micro = 0.429 +- 0.019


In [8]:
estimate_prod('random_best_ab_240422', validation_2_df)

Likes - 3319, Like Rate = 0.519 +- 0.022, Like Rate Micro = 0.433 +- 0.023


In [9]:
best_meme_from_each_source = BestMemeFromEachSource(meme_features_daily_df)
estimate(best_meme_from_each_source, validation_df)

Likes - 470, Like Rate = 0.588 +- 0.039, Like Rate Micro = 0.604 +- 0.040


In [10]:
best_meme_from_each_source = BestMemeFromEachSource(meme_features_daily_df)
estimate(best_meme_from_each_source, validation_2_df)

Likes - 118, Like Rate = 0.628 +- 0.076, Like Rate Micro = 0.619 +- 0.076


In [11]:
most_liked = MostLiked(meme_features_daily_df)
estimate(most_liked, validation_df)

Likes - 1402, Like Rate = 0.666 +- 0.042, Like Rate Micro = 0.568 +- 0.045


In [15]:
most_liked = MostLiked(meme_features_daily_df)
estimate(most_liked, validation_2_df)

Likes - 326, Like Rate = 0.620 +- 0.056, Like Rate Micro = 0.517 +- 0.059


In [13]:
most_liked_v2 = MostLiked(meme_features_daily_v2_df)
estimate(most_liked_v2, validation_df)

Likes - 2979, Like Rate = 0.647 +- 0.029, Like Rate Micro = 0.584 +- 0.025


In [14]:
most_liked_v2 = MostLiked(meme_features_daily_v2_df)
estimate(most_liked_v2, validation_2_df)

Likes - 518, Like Rate = 0.603 +- 0.038, Like Rate Micro = 0.577 +- 0.038
